**Instrucciones** 

Para este problema, deberás entregar, de manera individual, un informe en PDF que estudie las estadísticas de un robot de limpieza reactivo, así como el enlace al repositorio en Github del código desarrollado para esta actividad. El código debe ajustarse al estilo solicita en el siguiente documento.

Dado:

* Habitación de MxN espacios.

* Número de agentes.

* Porcentaje de celdas inicialmente sucias.

* Tiempo máximo de ejecución.

Realiza la siguiente simulación:

* Inicializa las celdas sucias (ubicaciones aleatorias).

* Todos los agentes empiezan en la celda [1,1].

* En cada paso de tiempo:

  * Si la celda está sucia, entonces aspira.

  * Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).

* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).

* Porcentaje de celdas limpias después del termino de la simulación.

* Número de movimientos realizados por todos los agentes.

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [ ]:
!pip install mesa
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

     |████████████████████████████████| 668 kB 5.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 


In [ ]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content,Vacuum):
                grid[x][y] = 2
            else:
                grid[x][y] = content.state 
    return grid


class Vacuum(Agent):
    '''
    Representa a un agente o una celda con estado sucio(1) o limpio (0)
    '''
    def __init__(self, unique_id, pos, model, moore=True):
        super().__init__(unique_id, model) 
        self.pos = pos

    def rand_move(self):
        # Pick the next cell from the adjacent cells.
        possible_moves = self.model.grid.get_neighborhood(self.pos, True, False)
        next_move = self.random.choice(possible_moves)

        current_cell = self.model.grid.get_cell_list_contents([next_move])
        vacuumExists = len([obj for obj in current_cell if isinstance(obj, Vacuum)])
        # Now move:
        if vacuumExists == 0:
          self.model.grid.move_agent(self, next_move)

        

    def step(self):
        current_cell = self.model.grid.get_cell_list_contents([self.pos])
        cell_objs = [obj for obj in current_cell if isinstance(obj, CellAgent)]
        
        # maquina de estados
        if cell_objs[0].state == 1:
          cell_objs[0].state = 0
          model.cleanedCells += 1
        else :
          self.rand_move()

  
class CellAgent(Agent):
    def __init__(self, unique_id, state,  model):
      super().__init__(unique_id, model)
      #Estado limpio o sucio
      self.state = state


class CleaningModel(Model):
    '''
    Define el modelo del robot aspirador.
    '''
    def __init__(self, M, N, num_vacuum, dirty_cells):
        self.grid = MultiGrid(M, N, False)
        self.cleanedCells = 0
        self.steps = 0
        self.clean_cells = (M*N - dirty_cells)
        self.schedule = SimultaneousActivation(self)


        # Generar celdas sucias
        for i in range (dirty_cells):
            d_curr = CellAgent((i+30,i+31), 1, self)
            self.schedule.add(d_curr) 

            blank_cell = self.grid.find_empty()
            self.grid.place_agent(d_curr, blank_cell)

        # Generar celdas limpias
        for j in range (self.clean_cells):
            c_curr = CellAgent((j+1000,j+1000), 0, self)
            self.schedule.add(c_curr) 

            blank_cell = self.grid.find_empty()
            self.grid.place_agent(c_curr, blank_cell)

        #genera los agentes aspiradora
        for k in range (num_vacuum):
            v_curr = Vacuum((k+45,k+41),(1,1),self)
            self.schedule.add(v_curr)

            self.grid.place_agent(v_curr,(1,1)) 

        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
    def all_clean(self):
        if self.cleanedCells == dirty_cells:
          return True
        else:
          return False


    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()
        self.steps += 1

A continuación corremos el modelo

In [ ]:
# Definimos el tamaño del Grid
M = 10
N = 15
num_vacuum = 45
perc_dirty_cells = .5
tiempo_ejecucion = 0.10

dirty_cells = int(M*N*perc_dirty_cells)
print("Celdas Sucias:", dirty_cells)

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = CleaningModel(M,N,num_vacuum, dirty_cells)
while ((time.time()-start_time) < tiempo_ejecucion and not model.all_clean()):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))
print('Pasos:', model.steps)
print('Porcentaje de celdas limpiadas:', int(model.cleanedCells*100 / (dirty_cells)),'%')

Celdas Sucias: 75
Tiempo de ejecución: 0:00:00.093091
Pasos: 115
Porcentaje de celdas limpiadas: 100 %


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='GnBu')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim